In [ ]:
"""
how does distribution of hyp values differ by symbol?

in this one, we'll graph the distributions of the argmaxes, rather than the raw values
"""
%matplotlib inline

import time
import json
import shutil
from collections import defaultdict

import h5py
import numpy as np
import matplotlib.pyplot as plt
import sklearn.metrics.cluster

import torch

def run(filename):
    shutil.copyfile(f'../{filename}', '/tmp/foo.h5')
    f = h5py.File('/tmp/foo.h5', 'r')

    hypotheses_train = torch.from_numpy(f['hypotheses_train'][:])
    dsrefs_train = torch.from_numpy(f['dsrefs_train'][:].astype(np.uint8)).long()
    resdicts = f['resdicts']
    
    meta = json.loads(f['meta'][0])
    ref = meta['ref']
    print(ref)

    U_pred = hypotheses_train.size(0)

    V = hypotheses_train.size(2)
    print('V', V)
    
    N = dsrefs_train.size(0)

    batch_size = 128

    sample_render_ids = 4
    num_render_ids = len(resdicts)

    render_start_id = 0

    ids_to_render = [num_render_ids - sample_render_ids]

    print('_N', batch_size * sample_render_ids)

    for render_id in ids_to_render:
        resdict = json.loads(resdicts[render_id])
        episode = resdict['episode']
        n_start = render_id * batch_size
        n_end = (render_id + sample_render_ids) * batch_size
        _hypotheses_train = hypotheses_train[:, n_start:n_end]

        _N = _hypotheses_train.size(1)

        i = 1
        num_cols = 10
        num_rows = (U_pred + num_cols - 1) // num_cols
        plt.figure(figsize=(20, 2 * num_rows))
        for u in range(U_pred):
            _preds = _hypotheses_train[u, :].max(dim=-1)[1]
            plt.subplot(num_rows, num_cols, i)
            plt.hist(_preds, bins=np.arange(V + 1) - 0.5)
            i += 1
        plt.suptitle(f'{ref} episode={episode}')
        plt.show()

    f.close()

filenames = """
../hists/hypprop_with_predictor_eec52_hughtok8_20180929_003559.h5
../hists/hypprop_with_predictor_eec53_hughtok9_20180929_031930.h5
../hists/hypprop_with_predictor_eec54_hughtok1_20180929_094956.h5
../hists/hypprop_with_predictor_eec55_hughtok2_20180929_095009.h5
../hists/hypprop_with_predictor_eec56_hughtok33_20180929_095031.h5
../hists/hypprop_with_predictor_eec57_hughtok4_20180929_095455.h5
"""

filenames = filenames.split('\n')
filenames = [f.replace('../', '') for f in filenames if f != '']

for filename in filenames:
    run(filename)